# Extending: policies, dynamics, and reports

This page shows how the base models fit together and how to add custom report elements that return tables.

## Base models in practice

- `Dataset` wraps a `SingleYearDataset` of entity tables (e.g., person, household) for one year.
- `Policy` and `Dynamics` hold parameter changes (`ParameterValue`) and optional `simulation_modifier` callables.
- `Simulation` ties them together and runs country-specific logic to populate `simulation.result`.
- `ReportElement` subclasses compute analyses and return a pandas DataFrame.

You can create a policy change by declaring a `Parameter` and a `ParameterValue`, then passing it to `Policy(parameter_values=[...])`. Dynamics work the same way.

In [1]:
from datetime import datetime
from policyengine.models import Policy, Dynamics, Parameter, ParameterValue
param = Parameter(name="demo.parameter", data_type=float)
pv = ParameterValue(parameter=param, model_version="1.0.0", start_date=datetime(2025,1,1), value=1.23)
policy = Policy(name="demo policy", parameter_values=[pv])
dynamics = Dynamics(name="static")


## Custom report elements

Subclass `ReportElement` and implement `run()` to return a table. Keep analyses table-first so results are easy to store and compare later.

Below, we build a tiny toy dataset with a household table containing `gov_balance` and `weight_value`. We then define a report element that computes weighted means by decile.

In [2]:
import pandas as pd
from dataclasses import dataclass
from policyengine.models import Dataset, Simulation, Report
from policyengine.models.single_year_dataset import SingleYearDataset
from policyengine.models.reports import ReportElement

# Build a small, realistic household table
household = pd.DataFrame({
    'household_id': [1,2,3,4,5,6,7,8,9,10],
    'gov_balance':   [-2000, -500, 0, 250, 500,  750,  1000,  2000,  4000, 8000],
    'weight_value':  [   10,   20, 5,  10, 25,   15,    5,    5,   10,    5],
})
toy = Dataset(
    name="toy",
    data=SingleYearDataset(tables={'household': household}, year=2024),
    dataset_type="uk",
)
sim = Simulation(dataset=toy, policy=policy, dynamics=dynamics, country="uk")
# For custom report elements, we can set a result directly when we already have tables
sim.result = toy
household.head()


,household_id,gov_balance,weight_value
0,1,-2000,10
1,2,-500,20
2,3,0,5
3,4,250,10
4,5,500,25


In [3]:
class BalanceByDecile(ReportElement):
    sim: Simulation

    def run(self) -> pd.DataFrame:
        df = self.sim.result.data.tables['household'][['gov_balance','weight_value']].copy()
        # Compute deciles on gov_balance
        df['decile'] = pd.qcut(df['gov_balance'], 10, labels=range(1,11), duplicates='drop')
        # Weighted mean by decile
        out = (
            df.groupby('decile').apply(
                lambda g: (g['gov_balance'] * g['weight_value']).sum() / g['weight_value'].sum()
            ).reset_index(name='mean_gov_balance')
        )
        return out

BalanceByDecile(name="balance by decile", sim=sim).run()


/var/folders/4b/b61z5j8n4x34kmv3zmtbqcgm0000gn/T/ipykernel_16239/3757811630.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('decile').apply(
/var/folders/4b/b61z5j8n4x34kmv3zmtbqcgm0000gn/T/ipykernel_16239/3757811630.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('decile').apply(


,decile,mean_gov_balance
0,1,-2000.0
1,2,-500.0
2,3,0.0
3,4,250.0
4,5,500.0
5,6,750.0
6,7,1000.0
7,8,2000.0
8,9,4000.0
9,10,8000.0


### Notes

- Keep report elements pure and table-focused; return a DataFrame you can store.
- You can combine multiple custom elements in a `Report` for batch runs.
- For built-in examples, see `AggregateChangeReportElement` which compares aggregates across simulations.